In [2]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt

In [3]:
TRAIN_PATH = './files/magdy/'
sample_rate, audio = wavfile.read(TRAIN_PATH + "15.wav")
print(sample_rate)
# Normalizing the audio "dividing its values on the max value"
def normalize_audio(audio):
    audio = audio / np.max(np.abs(audio))
    return audio

# Calling the normalize function
audio = normalize_audio(audio)

# Dividing the audio into frames due to "audio is not stationary"
def frame_audio(audio, FFT_size=2048, hop_size=512, sample_rate=44100):
    # hop_size in ms
    
    audio     = np.pad(audio, int(FFT_size / 2), mode='reflect')    # return => pad : ndarray Padded array of rank equal to array with shape increased according to pad_width
    audio = audio[:,0] #stereo to only one channel
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int) # return frame length
    frame_num = int((len(audio) - FFT_size) / frame_len) + 1
    frames    = np.zeros((frame_num, FFT_size))

    for n in range(frame_num):
        frames[n] = audio[n*frame_len : n*frame_len+1]
    
    return frames

hop_size = 512 # hop is the number of points intersected among frames   # if the hop size increased "exp 2", then the number of frames decrease
FFT_size = 2048

audio_framed = frame_audio(audio, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate)

window = get_window("hann", FFT_size, fftbins=True) # Create window, its size = frame size

audio_win = audio_framed * window    # Multiply window by frame to make edges sync to zero

audio_winT = np.transpose(audio_win) # Transpose the result

audio_fft = np.empty((int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F') # Getting Fast Fourier 

for n in range(audio_fft.shape[1]): 
    audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]] # Applyting fft to all frames

audio_fft = np.transpose(audio_fft)        # Transpose the Fourier

audio_power = np.square(np.abs(audio_fft)) # Calculating the signal power

freq_min = 0
freq_high = sample_rate / 2
mel_filter_num = 10

def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0) # mel equation to convert from freq scale to mel scale

def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0) # mel equation to convert from mel scale to freq scale

def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100): # getting filter points
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)

    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2) # created 12 mel banks
    freqs = met_to_freq(mels) # convert these points back to freq scale "center frequencies of the mel band"
    
    return np.floor((FFT_size + 1) / sample_rate * freqs).astype(int), freqs # round bins to nearest freq bin to 

filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=44100)

def get_filters(filter_points, FFT_size): # constructing triangle filters using filter points
    filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
    return filters

filters = get_filters(filter_points, FFT_size)

# taken from the librosa library
enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
filters *= enorm[:, np.newaxis]

audio_filtered = np.dot(filters, np.transpose(audio_power))
audio_log = 10.0 * np.log10(audio_filtered)

def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num,filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)
    
    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
    return basis

dct_filter_num = 40

dct_filters = dct(dct_filter_num, mel_filter_num)

cepstral_coefficents = np.dot(dct_filters, audio_log)

print(np.mean(cepstral_coefficents[:13].T, axis=0))
print(cepstral_coefficents[:13,0].T)
print(cepstral_coefficents.shape)

# #-----------------------------------------------------------------------------------------------------
# def mfcc(*, y=None, sr=22050, S=None, n_mfcc=20, dct_type=2, norm="ortho", lifter=0, **kwargs):
#     M = scipy.fftpack.dct(S, axis=-2, type=dct_type, norm=norm)[..., :n_mfcc, :]

#     if lifter > 0:
#         # shape lifter for broadcasting
#         LI = np.sin(np.pi * np.arange(1, 1 + n_mfcc, dtype=M.dtype) / lifter)
#         LI = util.expand_to(LI, ndim=S.ndim, axes=-2)

#         M *= 1 + (lifter / 2) * LI
#         return M
#     elif lifter == 0:
#         return M
#     else:
#         raise ParameterError("MFCC lifter={} must be a non-negative number".format(lifter))


44100
[-9.91017444e+02  1.38784362e+02  1.37784489e+02  1.27776973e+02
  1.18441726e+02  9.97819895e+01  8.37017364e+01  6.54920344e+01
  4.42850595e+01  2.10092922e+01  1.46949951e-12 -2.10092922e+01
 -4.42850595e+01]
[-1.06316731e+03  1.38107877e+02  1.37842866e+02  1.27103390e+02
  1.19440302e+02  1.00521628e+02  8.45090840e+01  6.53289367e+01
  4.49889737e+01  2.14151634e+01  1.56577918e-12 -2.14151634e+01
 -4.49889737e+01]
(40, 3)


In [7]:
"""Audio Framing¶
Because audio is a non stationary process, the FFT will produce distortions. To overcome this we can assume that the audio is a stationary process for a 
short periods of time. Because of that we devide the signal into short frames. Each audio frame will be the same size as the FFT. Also we want the frames to
overlap. We do that so that the frames will have some correlation between them and because we loose the information on the edges of each frame after applying
a window function."""

'Audio Framing¶\nBecause audio is a non stationary process, the FFT will produce distortions. To overcome this we can assume that the audio is a stationary process for a \nshort periods of time. Because of that we devide the signal into short frames. Each audio frame will be the same size as the FFT. Also we want the frames to\noverlap. We do that so that the frames will have some correlation between them and because we loose the information on the edges of each frame after applying\na window function.'

In [47]:
"""Convert to frequency domain
Now we will convert the audio, which is currently in the time domain, to frequency domain. The FFT assumes the audio to be periodic and continues.
By framing the signal we assured the audio to be periodic. To make the audio continues, we apply a window function on every frame. If we wont do that, We will
get high frequency distortions. To overcome this, we first need to apply a window function to the framed audio and then perforn FFT. The window assures that
both ends of the signal will end close to zero."""

'Convert to frequency domain\nNow we will convert the audio, which is currently in the time domain, to frequency domain. The FFT assumes the audio to be periodic and continues.\nBy framing the signal we assured the audio to be periodic. To make the audio continues, we apply a window function on every frame. If we wont do that, We will\nget high frequency distortions. To overcome this, we first need to apply a window function to the framed audio and then perforn FFT. The window assures that\nboth ends of the signal will end close to zero.'

In [78]:
# import librosa
# import numpy as np

# zcr_scratch = np.array

# x, sample_rate = librosa.load('./files/magdy/1.wav', res_type='kaiser_fast')

# for index in range(0, len(x) - 1):
#     zcr_scratch = zcr_scratch + (np.sign(x[index]) - np.sign(x[index + 1]) ) / 2

# print(zcr_scratch)